In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    #connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=mierda")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    #create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE accounts")
    cur.execute("CREATE DATABASE accounts")

    #close connection to default database
    conn.close()

    #connect to sparkify database
    conn= psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=mierda")
    cur = conn.cursor()

    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_tables_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_tables_queries:
        cur.execute(query)
        conn.commit()

In [6]:
AccountsCountry = pd.read_csv("Wealth-AccountsCountry.csv")
AccountData = pd.read_csv("Wealth-AccountData.csv")
AccountsSeries = pd.read_csv("Wealth-AccountSeries.csv")

In [26]:
AccountsCountry.columns

Index(['Code', 'Short Name', 'Long Name', 'Table Name', 'Currency Unit'], dtype='object')

In [7]:
AccountsCountry = AccountsCountry[['Code','Short Name', 'Long Name','Table Name', 'Currency Unit']].dropna()

In [8]:
AccountsCountry[AccountsCountry['Code']=='ARM']

,Code,Short Name,Long Name,Table Name,Currency Unit
2,ARM,Armenia,Republic of Armenia,Armenia,Armenian dram


In [10]:
AccountData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '2005 [YR2005]', '2010 [YR2010]', '2015 [YR2015]',
       '2018 [YR2018]'],
      dtype='object')

In [9]:
pd.options.display.float_format = '{:.0f}'.format

In [10]:
AccountData=AccountData[['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '2005 [YR2005]','2010 [YR2010]','2015 [YR2015]']].dropna()

In [11]:
AccountData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7592 entries, 0 to 7591
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   7592 non-null   object 
 1   Country Code   7592 non-null   object 
 2   Series Name    7592 non-null   object 
 3   Series Code    7592 non-null   object 
 4   2005 [YR2005]  7592 non-null   float64
 5   2010 [YR2010]  7592 non-null   float64
 6   2015 [YR2015]  7592 non-null   float64
dtypes: float64(3), object(4)
memory usage: 415.3+ KB


In [12]:
AccountsSeries = AccountsSeries[['Code', 'Indicator Name', 'Long definition','Topic']]

In [13]:
cur, conn = create_database()

In [14]:
songplay_table_create = ("""CREATE TABLE IF NOT EXISTS accountscountry(
code VARCHAR PRIMARY KEY,
short_name VARCHAR,
long_name VARCHAR,
table_name VARCHAR,
currency_unit VARCHAR
)""")

In [15]:
cur.execute(songplay_table_create)
conn.commit()

In [16]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
series_name VARCHAR,
series_code VARCHAR,
year_1995 numeric,
year_2005 numeric,
year_2010 numeric,
year_2015 numeric,
year_2018 numeric
)""")
cur.execute(accounts_data_table_create)
conn.commit()

In [17]:
accountseries_data_table_create= ("""CREATE TABLE IF NOT EXISTS accountseries(
code VARCHAR,
indicator_name VARCHAR,
long_definition VARCHAR,
topic VARCHAR
)""")
cur.execute(accountseries_data_table_create)
conn.commit()

In [18]:
accounts_country_table_insert= ("""INSERT INTO accountscountry(
code,
short_name ,
long_name ,
table_name ,
currency_unit )
VALUES(%s, %s, %s, %s, %s)""")

In [19]:
for i, row in AccountsCountry.iterrows():
    cur.execute(accounts_country_table_insert, list(row))

In [20]:
conn.commit()

In [21]:
accounts_data_table_insert = ("""INSERT INTO accountsdata(
country_name,
country_code,
series_name,
series_code,
year_2005,
year_2010,
year_2015)
VALUES(%s, %s, %s, %s, %s, %s, %s)
""")

In [22]:
for i, row in AccountData.iterrows():
    cur.execute(accounts_data_table_insert, list(row))

In [23]:
conn.commit()

In [24]:
accountseries_data_table_insert = ("""INSERT INTO accountseries(
code,
indicator_name,
long_definition,
topic
)VALUES(%s, %s, %s, %s)
""")

In [25]:
for i, row in AccountsSeries.iterrows():
    cur.execute(accountseries_data_table_insert, list(row))

In [26]:
conn.commit()

In [27]:
cur.close()
conn.close()